In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# dataset = pd.read_csv('msc_data_cleaned_1.csv')
# dataset = pd.read_csv('msc_data_cleaned.csv')
# dataset = pd.read_excel("cleaned_data3.xlsx")
dataset = pd.read_csv('msc_data_cleaned_new1.csv')

In [2]:
dataset

,package,gender,age_cat,con_age,language_id,total_voice_usage_min,total_data_usage_mb,device_type,total_revenue,vas_revenue
0,MQSM,0,51-60,7-10 months,2002,76.25,12177.65,SMART,1319.92,226.67
1,CRBT_SRV,1,41-45,1-2 years,2002,227.42,4502.36,NaN,448.00,29.42
2,CRBT_SRV,0,31-35,2-3 years,2078,143.83,7318.22,SMART,1357.65,143.43
3,MQSM,1,41-45,10-12 months,2002,128.55,0.21,SMART,277.25,154.28
4,CRBT_SRV,1,46-50,1-2 years,2002,15.27,20411.27,SMART,1797.99,190.03
...,...,...,...,...,...,...,...,...,...,...
26275,CRBT_SRV,1,26-30,more than 5 years,2078,850.62,0.21,NaN,690.41,150.79
26276,CRBT_SRV,1,51-60,more than 5 years,2078,223.58,0.00,FEATURE,365.07,158.15
26277,CRBT_SRV,1,51-60,more than 5 years,2078,616.03,0.00,BASIC,884.65,115.15
26278,CRBT_SRV,0,51-60,3-5 years,2002,474.97,494.23,SMART,661.97,180.52


In [3]:
def clean_age(x):
    if '51-60' in x or '60+' in x:
        return '51-60'
    if '41-45' in x or '46-50' in x:
        return '41-50'
    if '31-35' in x or '36-40' in x:
        return '31-40'
    if '21-25' in x or '26-30' in x:
        return '21-30'
    return 'Under 20'

dataset['age_cat'] = dataset['age_cat'].apply(clean_age)

In [4]:
dataset = dataset.dropna(subset=['device_type'])
dataset = dataset.dropna(subset=['language_id'])

In [5]:
def clean_deviceType(x):
    if 'BASIC' in x:
        return 'BASIC'
    if 'SMART' in x:
        return 'SMART'
    if 'FEATURE' in x:
        return 'FEATURE'

dataset['device_type'] = dataset['device_type'].apply(clean_deviceType)

In [6]:
sin= 2078.0
eng = 2002.0
# tam = 2079.0
def clean_language(x):
    if sin in [x]:
        return 'Sinhala'
    if eng in [x]:
        return 'English'
    return 'Tamil'

dataset['language_id'] = dataset['language_id'].apply(clean_language)

In [7]:
def clean_connectionAge(x):
    if 'more than 5 years' in x:
        return 'more than 5 years'
    if '3-5 years' in x:
        return '3-5 years'
    if '2-3 years' in x or '1-2 years' in x:
        return '1-3 years'
    return 'less than 1'

dataset['con_age'] = dataset['con_age'].apply(clean_connectionAge)

In [8]:
from sklearn.preprocessing import LabelEncoder

dataTransform = dataset.copy()

In [9]:
le_language = LabelEncoder()
dataTransform['language_id'] = le_language.fit_transform(dataset['language_id'])
le_age = LabelEncoder()
dataTransform['age_cat'] = le_age.fit_transform(dataset['age_cat'])
le_conAge = LabelEncoder()
dataTransform['con_age'] = le_conAge.fit_transform(dataset['con_age'])
le_device = LabelEncoder()
dataTransform['device_type'] = le_device.fit_transform(dataset['device_type'])
le_package = LabelEncoder()
dataTransform['package'] = le_package.fit_transform(dataset['package'])
dataTransform["package"].unique()


array([2, 1, 0])

In [10]:
dataTransform.shape

(21781, 10)

In [11]:
dtf = dataTransform['package']
# Min-Max Normalization
df = dataTransform.drop('package', axis=1)
df_norm = (df-df.min())/(df.max()-df.min())

df_norm.head()

,gender,age_cat,con_age,language_id,total_voice_usage_min,total_data_usage_mb,device_type,total_revenue,vas_revenue
0,0.0,0.75,0.666667,0.0,0.008460,0.098454,1.0,0.019094,0.003322
2,0.0,0.25,0.000000,0.5,0.015958,0.059166,1.0,0.019646,0.002089
3,1.0,0.50,0.666667,0.0,0.014263,0.000002,1.0,0.003819,0.002250
4,1.0,0.50,0.000000,0.0,0.001694,0.165021,1.0,0.026097,0.002780
6,0.0,0.75,0.666667,0.0,0.091588,0.000000,0.5,0.017776,0.001853


In [12]:
def outliers(df,feature):
    Q1 = df[feature].quantile(0.05)
    Q3 = df[feature].quantile(0.95)
    IQR = Q3 - Q1
    
    lowerBound = Q1 - 1.5*IQR
    uperBound  = Q3 + 1.5*IQR
    
    ls = df.index[(df[feature]<lowerBound) | (df[feature]>uperBound)]
    return ls

In [13]:
index_list = []
for f in ['total_voice_usage_min','total_revenue']:
    index_list.extend(outliers(df_norm,f))

In [14]:
def remove(df,ls):
    ls = sorted(set(ls))
    df =df.drop(ls)
    return df

In [15]:
cleand_df = remove(df_norm,index_list)

In [16]:
cleand_df.shape

(21697, 9)

In [17]:
result = pd.concat([cleand_df, dtf], axis=1, join='inner')
display(result)

,gender,age_cat,con_age,language_id,total_voice_usage_min,total_data_usage_mb,device_type,total_revenue,vas_revenue,package
0,0.0,0.75,0.666667,0.0,0.008460,9.845378e-02,1.0,0.019094,0.003322,2
2,0.0,0.25,0.000000,0.5,0.015958,5.916630e-02,1.0,0.019646,0.002089,1
3,1.0,0.50,0.666667,0.0,0.014263,1.697807e-06,1.0,0.003819,0.002250,2
4,1.0,0.50,0.000000,0.0,0.001694,1.650209e-01,1.0,0.026097,0.002780,1
6,0.0,0.75,0.666667,0.0,0.091588,0.000000e+00,0.5,0.017776,0.001853,1
...,...,...,...,...,...,...,...,...,...,...
26274,0.0,0.75,1.000000,0.0,0.035557,2.091892e-02,1.0,0.014661,0.002627,0
26276,1.0,0.75,1.000000,0.5,0.024807,0.000000e+00,0.5,0.005106,0.002307,1
26277,1.0,0.75,1.000000,0.5,0.068350,0.000000e+00,0.0,0.012717,0.001670,1
26278,0.0,0.75,0.333333,0.0,0.052699,3.995747e-03,1.0,0.009455,0.002639,1


In [18]:
y = np.array(result.iloc[:,9])
X = np.array(result.iloc[:,0:9])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [19]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
X, y=smote.fit_resample(X_train, y_train)

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model=RandomForestClassifier(n_estimators=100)
rf = model.fit(X_train,y_train)
y_pred=model.predict(X_test)

In [21]:
from sklearn import metrics

print("Accuracy:",(metrics.accuracy_score(y_test, y_pred))*100)


Accuracy: 74.65437788018433


In [22]:
import pickle
data = {"model": rf, "le_age": le_age, "le_device":le_device, "le_language":le_language, "le_conAge":le_conAge,
        "le_package":le_package}
with open('randomForest1.pkl', 'wb') as file:
    pickle.dump(data, file)
